In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import ttest_1samp, shapiro
from sklearn.feature_selection import chi2

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.metrics import recall_score, make_scorer, roc_auc_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from graphviz import Source
from sklearn.tree import export_graphviz
from sklearn import tree
from IPython.display import SVG, display
# import shap

import warnings
warnings.filterwarnings('ignore')
from scipy.stats.mstats import winsorize
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
import pandas as pd
from sklearn.preprocessing import PowerTransformer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Old

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ml1_final_project/data_cardiovascular_risk.csv')
columns_to_convert = ['sex', 'education', 'is_smoking', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'TenYearCHD']
df[columns_to_convert] = df[columns_to_convert].astype('object')
categorical_columns = df.select_dtypes(include='object').columns
numerical_columns = df.select_dtypes(include=['float', 'int']).columns

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Remove rows where 'bpmeds' column is null
df = df.dropna(subset=['BPMeds'])

imputing_median1 = df[df['is_smoking'] == 'YES']['cigsPerDay'].median()
df['cigsPerDay'] = df['cigsPerDay'].fillna(imputing_median1)

df.drop('id', axis=1, inplace=True)
df.drop('education', axis=1, inplace=True)
df.drop(['is_smoking', 'prevalentStroke'], axis=1, inplace=True)

for var in ['totChol', 'BMI', 'heartRate']:
  imputer = IterativeImputer(random_state=0)
  df[[var]] = imputer.fit_transform(df[[var]])


# fit and transform the imputer on your data
imputer = IterativeImputer(random_state = 0)
imputer.fit(df[['glucose']])

df[['glucose']] = imputer.transform(df[['glucose']])


df = pd.get_dummies(df, drop_first=True)

X = df.drop(['TenYearCHD_1'], axis = 1)
y = df['TenYearCHD_1']

X_winsorized = X.copy()


# Apply winsorization to each column with the optimal limits
X_winsorized = X.copy()

X_winsorized['age'] = winsorize(X_winsorized['age'], limits=(0, 0))
X_winsorized['cigsPerDay'] = winsorize(X_winsorized['cigsPerDay'], limits=(0.7, 0.01))
X_winsorized['totChol'] = winsorize(X_winsorized['totChol'], limits=(0, 0))
X_winsorized['sysBP'] = winsorize(X_winsorized['sysBP'], limits=(0, 0))
X_winsorized['diaBP'] = winsorize(X_winsorized['diaBP'], limits=(0.8, 0))
X_winsorized['BMI'] = winsorize(X_winsorized['BMI'], limits=(0, 0))
X_winsorized['heartRate'] = winsorize(X_winsorized['heartRate'], limits=(0, 0))
X_winsorized['glucose'] = winsorize(X_winsorized['glucose'], limits=(0.7, 0.2))
X_winsorized['sex_M'] = winsorize(X_winsorized['sex_M'], limits=(0, 0))
X_winsorized['BPMeds_1.0'] = winsorize(X_winsorized['BPMeds_1.0'], limits=(0, 0))
X_winsorized['prevalentHyp_1'] = winsorize(X_winsorized['prevalentHyp_1'], limits=(0, 0.4))
X_winsorized['diabetes_1'] = winsorize(X_winsorized['diabetes_1'], limits=(0, 0))

# Split the winsorized data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_winsorized, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

# Train Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train_s, y_train)

# Make predictions and calculate AUC
y_pred = gnb.predict_proba(X_test_s)[:, 1]
auc = roc_auc_score(y_test, y_pred)

# Print the AUC score
print("AUC:", auc)


AUC: 0.7453729977116705


Smote 0.2 OverSampling 0.5 UnderSampling


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score

df = pd.read_csv('/content/drive/MyDrive/ml1_final_project/data_cardiovascular_risk.csv')
columns_to_convert = ['sex', 'education', 'is_smoking', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'TenYearCHD']
df[columns_to_convert] = df[columns_to_convert].astype('object')
categorical_columns = df.select_dtypes(include='object').columns
numerical_columns = df.select_dtypes(include=['float', 'int']).columns

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Remove rows where 'bpmeds' column is null
df = df.dropna(subset=['BPMeds'])

imputing_median1 = df[df['is_smoking'] == 'YES']['cigsPerDay'].median()
df['cigsPerDay'] = df['cigsPerDay'].fillna(imputing_median1)

df.drop('id', axis=1, inplace=True)
df.drop('education', axis=1, inplace=True)
df.drop(['is_smoking', 'prevalentStroke'], axis=1, inplace=True)

for var in ['totChol', 'BMI', 'heartRate']:
  imputer = IterativeImputer(random_state=0)
  df[[var]] = imputer.fit_transform(df[[var]])


# fit and transform the imputer on your data
imputer = IterativeImputer(random_state = 0)
imputer.fit(df[['glucose']])

df[['glucose']] = imputer.transform(df[['glucose']])


df = pd.get_dummies(df, drop_first=True)

X = df.drop(['TenYearCHD_1'], axis = 1)
y = df['TenYearCHD_1']

X_winsorized = X.copy()


# Apply winsorization to each column with the optimal limits
X_winsorized = X.copy()

X_winsorized['age'] = winsorize(X_winsorized['age'], limits=(0, 0))
X_winsorized['cigsPerDay'] = winsorize(X_winsorized['cigsPerDay'], limits=(0.7, 0.01))
X_winsorized['totChol'] = winsorize(X_winsorized['totChol'], limits=(0, 0))
X_winsorized['sysBP'] = winsorize(X_winsorized['sysBP'], limits=(0, 0))
X_winsorized['diaBP'] = winsorize(X_winsorized['diaBP'], limits=(0.8, 0))
X_winsorized['BMI'] = winsorize(X_winsorized['BMI'], limits=(0, 0))
X_winsorized['heartRate'] = winsorize(X_winsorized['heartRate'], limits=(0, 0))
X_winsorized['glucose'] = winsorize(X_winsorized['glucose'], limits=(0.7, 0.2))
X_winsorized['sex_M'] = winsorize(X_winsorized['sex_M'], limits=(0, 0))
X_winsorized['BPMeds_1.0'] = winsorize(X_winsorized['BPMeds_1.0'], limits=(0, 0))
X_winsorized['prevalentHyp_1'] = winsorize(X_winsorized['prevalentHyp_1'], limits=(0, 0.4))
X_winsorized['diabetes_1'] = winsorize(X_winsorized['diabetes_1'], limits=(0, 0))



# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply MinMaxScaler to scale the features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE for oversampling and RandomUnderSampler for undersampling
smote = SMOTE(sampling_strategy=0.2)
undersampler = RandomUnderSampler(sampling_strategy=0.5)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train_resampled, y_train_resampled)

# Train Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train_resampled, y_train_resampled)

# Make predictions on test data
y_pred_proba = gnb.predict_proba(X_test_scaled)[:, 1]

# Calculate AUC score
auc = roc_auc_score(y_test, y_pred_proba)
print("AUC:", auc)


AUC: 0.7201281464530893


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score

df = pd.read_csv('/content/drive/MyDrive/ml1_final_project/data_cardiovascular_risk.csv')
columns_to_convert = ['sex', 'education', 'is_smoking', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'TenYearCHD']
df[columns_to_convert] = df[columns_to_convert].astype('object')
categorical_columns = df.select_dtypes(include='object').columns
numerical_columns = df.select_dtypes(include=['float', 'int']).columns

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Remove rows where 'bpmeds' column is null
df = df.dropna(subset=['BPMeds'])

imputing_median1 = df[df['is_smoking'] == 'YES']['cigsPerDay'].median()
df['cigsPerDay'] = df['cigsPerDay'].fillna(imputing_median1)

df.drop('id', axis=1, inplace=True)
df.drop('education', axis=1, inplace=True)
df.drop(['is_smoking', 'prevalentStroke'], axis=1, inplace=True)

for var in ['totChol', 'BMI', 'heartRate']:
  imputer = IterativeImputer(random_state=0)
  df[[var]] = imputer.fit_transform(df[[var]])


# fit and transform the imputer on your data
imputer = IterativeImputer(random_state = 0)
imputer.fit(df[['glucose']])

df[['glucose']] = imputer.transform(df[['glucose']])


df = pd.get_dummies(df, drop_first=True)

X = df.drop(['TenYearCHD_1'], axis = 1)
y = df['TenYearCHD_1']

X_winsorized = X.copy()


# Apply winsorization to each column with the optimal limits
X_winsorized = X.copy()

X_winsorized['age'] = winsorize(X_winsorized['age'], limits=(0, 0))
X_winsorized['cigsPerDay'] = winsorize(X_winsorized['cigsPerDay'], limits=(0.7, 0.01))
X_winsorized['totChol'] = winsorize(X_winsorized['totChol'], limits=(0, 0))
X_winsorized['sysBP'] = winsorize(X_winsorized['sysBP'], limits=(0, 0))
X_winsorized['diaBP'] = winsorize(X_winsorized['diaBP'], limits=(0.8, 0))
X_winsorized['BMI'] = winsorize(X_winsorized['BMI'], limits=(0, 0))
X_winsorized['heartRate'] = winsorize(X_winsorized['heartRate'], limits=(0, 0))
X_winsorized['glucose'] = winsorize(X_winsorized['glucose'], limits=(0.7, 0.2))
X_winsorized['sex_M'] = winsorize(X_winsorized['sex_M'], limits=(0, 0))
X_winsorized['BPMeds_1.0'] = winsorize(X_winsorized['BPMeds_1.0'], limits=(0, 0))
X_winsorized['prevalentHyp_1'] = winsorize(X_winsorized['prevalentHyp_1'], limits=(0, 0.4))
X_winsorized['diabetes_1'] = winsorize(X_winsorized['diabetes_1'], limits=(0, 0))



# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply MinMaxScaler to scale the features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE for oversampling and RandomUnderSampler for undersampling
smote = SMOTE(sampling_strategy=0.4)
undersampler = RandomUnderSampler(sampling_strategy=0.5)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train_resampled, y_train_resampled)

# Train Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train_resampled, y_train_resampled)

# Make predictions on test data
y_pred_proba = gnb.predict_proba(X_test_scaled)[:, 1]

# Calculate AUC score
auc = roc_auc_score(y_test, y_pred_proba)
print("AUC:", auc)


AUC: 0.7113226544622425


Old with Yeo Johnson before removing outliers

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ml1_final_project/data_cardiovascular_risk.csv')
columns_to_convert = ['sex', 'education', 'is_smoking', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'TenYearCHD']
df[columns_to_convert] = df[columns_to_convert].astype('object')
categorical_columns = df.select_dtypes(include='object').columns
numerical_columns = df.select_dtypes(include=['float', 'int']).columns

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Remove rows where 'bpmeds' column is null
df = df.dropna(subset=['BPMeds'])

imputing_median1 = df[df['is_smoking'] == 'YES']['cigsPerDay'].median()
df['cigsPerDay'] = df['cigsPerDay'].fillna(imputing_median1)

df.drop('id', axis=1, inplace=True)
df.drop('education', axis=1, inplace=True)
df.drop(['is_smoking', 'prevalentStroke'], axis=1, inplace=True)

for var in ['totChol', 'BMI', 'heartRate']:
  imputer = IterativeImputer(random_state=0)
  df[[var]] = imputer.fit_transform(df[[var]])


# fit and transform the imputer on your data
imputer = IterativeImputer(random_state = 0)
imputer.fit(df[['glucose']])

df[['glucose']] = imputer.transform(df[['glucose']])


#### Applying Yeo-Johnson
# Filter numerical columns of type 'int' and 'float'
numerical_cols = df.select_dtypes(include=['int', 'float']).columns

# Apply Yeo-Johnson transformation
transformer = PowerTransformer(method='yeo-johnson')
df[numerical_cols] = transformer.fit_transform(df[numerical_cols])
##########################################


df = pd.get_dummies(df, drop_first=True)

X = df.drop(['TenYearCHD_1'], axis = 1)
y = df['TenYearCHD_1']

X_winsorized = X.copy()


# Apply winsorization to each column with the optimal limits
X_winsorized = X.copy()

X_winsorized['age'] = winsorize(X_winsorized['age'], limits=(0, 0))
X_winsorized['cigsPerDay'] = winsorize(X_winsorized['cigsPerDay'], limits=(0.7, 0.01))
X_winsorized['totChol'] = winsorize(X_winsorized['totChol'], limits=(0, 0))
X_winsorized['sysBP'] = winsorize(X_winsorized['sysBP'], limits=(0, 0))
X_winsorized['diaBP'] = winsorize(X_winsorized['diaBP'], limits=(0.8, 0))
X_winsorized['BMI'] = winsorize(X_winsorized['BMI'], limits=(0, 0))
X_winsorized['heartRate'] = winsorize(X_winsorized['heartRate'], limits=(0, 0))
X_winsorized['glucose'] = winsorize(X_winsorized['glucose'], limits=(0.7, 0.2))
X_winsorized['sex_M'] = winsorize(X_winsorized['sex_M'], limits=(0, 0))
X_winsorized['BPMeds_1.0'] = winsorize(X_winsorized['BPMeds_1.0'], limits=(0, 0))
X_winsorized['prevalentHyp_1'] = winsorize(X_winsorized['prevalentHyp_1'], limits=(0, 0.4))
X_winsorized['diabetes_1'] = winsorize(X_winsorized['diabetes_1'], limits=(0, 0))

# Split the winsorized data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_winsorized, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

# Train Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train_s, y_train)

# Make predictions and calculate AUC
y_pred = gnb.predict_proba(X_test_s)[:, 1]
auc = roc_auc_score(y_test, y_pred)

# Print the AUC score
print("AUC:", auc)


AUC: 0.743304347826087


Old Yeo Johnson after outliers

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ml1_final_project/data_cardiovascular_risk.csv')
columns_to_convert = ['sex', 'education', 'is_smoking', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'TenYearCHD']
df[columns_to_convert] = df[columns_to_convert].astype('object')
categorical_columns = df.select_dtypes(include='object').columns
numerical_columns = df.select_dtypes(include=['float', 'int']).columns

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Remove rows where 'bpmeds' column is null
df = df.dropna(subset=['BPMeds'])

imputing_median1 = df[df['is_smoking'] == 'YES']['cigsPerDay'].median()
df['cigsPerDay'] = df['cigsPerDay'].fillna(imputing_median1)

df.drop('id', axis=1, inplace=True)
df.drop('education', axis=1, inplace=True)
df.drop(['is_smoking', 'prevalentStroke'], axis=1, inplace=True)

for var in ['totChol', 'BMI', 'heartRate']:
  imputer = IterativeImputer(random_state=0)
  df[[var]] = imputer.fit_transform(df[[var]])


# fit and transform the imputer on your data
imputer = IterativeImputer(random_state = 0)
imputer.fit(df[['glucose']])

df[['glucose']] = imputer.transform(df[['glucose']])





df = pd.get_dummies(df, drop_first=True)

X = df.drop(['TenYearCHD_1'], axis = 1)
y = df['TenYearCHD_1']

X_winsorized = X.copy()


# Apply winsorization to each column with the optimal limits
X_winsorized = X.copy()

X_winsorized['age'] = winsorize(X_winsorized['age'], limits=(0, 0))
X_winsorized['cigsPerDay'] = winsorize(X_winsorized['cigsPerDay'], limits=(0.7, 0.01))
X_winsorized['totChol'] = winsorize(X_winsorized['totChol'], limits=(0, 0))
X_winsorized['sysBP'] = winsorize(X_winsorized['sysBP'], limits=(0, 0))
X_winsorized['diaBP'] = winsorize(X_winsorized['diaBP'], limits=(0.8, 0))
X_winsorized['BMI'] = winsorize(X_winsorized['BMI'], limits=(0, 0))
X_winsorized['heartRate'] = winsorize(X_winsorized['heartRate'], limits=(0, 0))
X_winsorized['glucose'] = winsorize(X_winsorized['glucose'], limits=(0.7, 0.2))
X_winsorized['sex_M'] = winsorize(X_winsorized['sex_M'], limits=(0, 0))
X_winsorized['BPMeds_1.0'] = winsorize(X_winsorized['BPMeds_1.0'], limits=(0, 0))
X_winsorized['prevalentHyp_1'] = winsorize(X_winsorized['prevalentHyp_1'], limits=(0, 0.4))
X_winsorized['diabetes_1'] = winsorize(X_winsorized['diabetes_1'], limits=(0, 0))

#### Applying Yeo-Johnson
# Filter numerical columns of type 'int' and 'float'
numerical_cols = X_winsorized.select_dtypes(include=['int', 'float']).columns

# Apply Yeo-Johnson transformation
transformer = PowerTransformer(method='yeo-johnson')
X_winsorized[numerical_cols] = transformer.fit_transform(X_winsorized[numerical_cols])
##########################################

# Split the winsorized data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_winsorized, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

# Train Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train_s, y_train)

# Make predictions and calculate AUC
y_pred = gnb.predict_proba(X_test_s)[:, 1]
auc = roc_auc_score(y_test, y_pred)

# Print the AUC score
print("AUC:", auc)


AUC: 0.7385629290617848


In [ ]:
X_winsorized


,age,cigsPerDay,totChol,sysBP,diaBP,BMI,heartRate,glucose,sex_M,BPMeds_1.0,prevalentHyp_1,diabetes_1
0,64,15.0,221.0,148.0,92.0,25.801918,90.0,84.0,0,0,0,0
1,36,15.0,212.0,168.0,98.0,29.770000,72.0,84.0,1,0,0,0
2,46,15.0,250.0,116.0,92.0,20.350000,88.0,88.0,0,0,0,0
3,50,20.0,233.0,158.0,92.0,28.260000,68.0,88.0,1,0,0,0
4,64,30.0,241.0,136.5,92.0,26.420000,70.0,84.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3384,61,15.0,217.0,182.0,92.0,26.980000,105.0,88.0,0,0,0,0
3385,60,15.0,261.0,123.5,92.0,29.280000,70.0,88.0,0,0,0,0
3386,46,15.0,199.0,102.0,92.0,21.960000,80.0,84.0,0,0,0,0
3387,44,15.0,352.0,164.0,119.0,28.920000,73.0,84.0,1,0,0,0
